## extracting from recon3d

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [2]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core= load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
glycolysis_ppp_tca_etc= load_json_cobra_model(filename=os.path.join(model_dir,"glycolysis_ppp_tca_etc_model.json"))



Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-02


In [3]:
glycolysis_ppp_tca_etc.reactions.NADPHM

Reaction identifier,NADPHM
Name,
Memory address,0x013984fc9670
Stoichiometry,nadph_c --> h_c + nadp_c Nicotinamide adenine dinucleotide phosphate - reduced --> H+ + Nicotinamide adenine dinucleotide phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [4]:
_REQUIRED_REACTION_ATTRIBUTES = [
    "id",
    "name",
    "metabolites",
    "lower_bound",
    "upper_bound",
#     "gene_reaction_rule",
]

_REQUIRED_METABOLITE_ATTRIBUTES = ["id", "name", "charge","formula", "compartment"]
_ORDERED_OPTIONAL_METABOLITE_KEYS = [
   
    "_bound",
    "notes",
    "annotation",
]


In [5]:
# def metabolite_to_dict(metabolite):
#     new_met = dict()
#     for key in _REQUIRED_METABOLITE_ATTRIBUTES:
#         new_met[key] = _fix_type(getattr(metabolite, key))
#     return new_met
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [6]:
reaction_list = []
reaction_list = [
                'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
                
                'PGMT',
                "GLPASE2",
                'GLDBRAN',
                'GLBRAN',
                "GLPASE1",
                'GLGNS1',
                'GGNG',
                'GALUi',

                'ADK1',
                ]

In [7]:
# reaction_list=[]
for rid in glycolysis_ppp_tca_etc.reactions:
    if rid.id not in reaction_list:
        reaction_list.append(rid.id)
len(reaction_list)

78

In [8]:
metabolite_list=[]
met_df=pd.DataFrame()
rxn_df=pd.DataFrame()
reactions_not_in_r3d=[]
for rxn in reaction_list:
    if rxn in R3D.reactions:
        if rxn == "DGAT":
            reaction = core.reactions.get_by_id(rxn)
            r=reaction_to_dict(reaction)
            reactions_not_in_r3d.append(rxn)
            print("ahaha gochu")
        else:
            reaction= R3D.reactions.get_by_id(rxn)
            r=reaction_to_dict(reaction)    
    elif rxn in core.reactions:
        reaction = core.reactions.get_by_id(rxn)
        r=reaction_to_dict(reaction)
        reactions_not_in_r3d.append(rxn)
    else:
        reaction = glycolysis_ppp_tca_etc.reactions.get_by_id(rxn)
        r=reaction_to_dict(reaction)
        reactions_not_in_r3d.append(rxn)


    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df= pd.concat([rxn_df, df])
    for met in reaction.metabolites:
        if met.id not in metabolite_list:
            if met.id in R3D.metabolites:
                metabolite= R3D.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)

            else:
                metabolite = core.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)
            # print(met.id)
            metabolite_list.append(met.id)
            df_2=pd.DataFrame.from_dict(m,orient='index')
            df_2=df_2.T
            met_df= pd.concat([met_df, df_2])


rxn_df=rxn_df.set_index('id')
met_df=met_df.set_index('id')
rxn_df
reactions_not_in_r3d

['PEPtm',
 'Htm',
 'PItm',
 'EX_co2_c',
 'EX_o2_c',
 'EX_h_c',
 'EX_h2o_c',
 'EX_pi_c',
 'EX_glc__D_c',
 'EX_lac__L_c',
 'NADPHM',
 'EX_r5p_c',
 'EX_o2s_c']

In [9]:
len(glycolysis_ppp_tca_etc.reactions)

69

In [10]:
len(rxn_df)

78

In [11]:
met_df

,name,charge,formula,compartment
id,,,,
pep_c,Phosphoenolpyruvate,-3,C3H2O6P,c
pep_m,Phosphoenolpyruvate,-3,C3H2O6P,m
pi_c,Phosphate,0,HO4P,c
pi_m,Phosphate,0,HO4P,m
co2_m,CO2 CO2,0,CO2,m
...,...,...,...,...
e4p_c,D-Erythrose 4-phosphate,-2,C4H7O7P,c
h2o2_c,Hydrogen peroxide,0,H2O2,c
o2s_c,Superoxide anion,-1,O2,c


In [12]:
rxn_df

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,notes,annotation
id,,,,,,,,
PEPtm,,"{'pep_c': 1, 'pep_m': -1, 'pi_c': -1, 'pi_m': 1}",0.0,1000.0,,NaN,NaN,NaN
PEPCKm,Phosphoenolpyruvate carboxykinase (GTP),"{'co2_m': 1.0, 'gdp_m': 1.0, 'gtp_m': -1.0, 'o...",0.0,1000.0,5106_AT1 or 5105_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PEPCKm']},"{'bigg.reaction': ['PEPCKm'], 'biocyc': ['META..."
PYK,Pyruvate kinase,"{'adp_c': -1.0, 'atp_c': 1.0, 'h_c': -1.0, 'pe...",0.0,1000.0,5315_AT2 or 5313_AT2 or 5315_AT3 or 5313_AT1 o...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PYK']},"{'bigg.reaction': ['PYK'], 'biocyc': ['META:PE..."
PCm,Pyruvate carboxylase,"{'adp_m': 1.0, 'atp_m': -1.0, 'h_m': 1.0, 'hco...",0.0,1000.0,5091_AT1 or 5091_AT2,Pyruvate metabolism,{'original_bigg_ids': ['PCm']},"{'bigg.reaction': ['PCm'], 'biocyc': ['META:PY..."
LDH_L,L-lactate dehydrogenase,"{'h_c': 1.0, 'lac__L_c': -1.0, 'nad_c': -1.0, ...",-1000.0,1000.0,(3945_AT1 and 3939_AT1) or 160287_AT1 or 3948_...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['LDH_L']},"{'bigg.reaction': ['LDH_L'], 'biocyc': ['META:..."
...,...,...,...,...,...,...,...,...
GTHPi,Glutathione peridoxase,"{'gthox_c': 1.0, 'gthrd_c': -2.0, 'h2o2_c': -1...",0.0,1000.0,7001_AT3 or 5052_AT3 or 2877_AT1 or 2876_AT2 o...,Glutathione metabolism,{'original_bigg_ids': ['GTHP']},"{'bigg.reaction': ['GTHPi'], 'biocyc': ['META:..."
GTHOr,Glutathione oxidoreductase,"{'gthox_c': -1.0, 'gthrd_c': 2.0, 'h_c': -1.0,...",0.0,1000.0,2936_AT1,Glutamate metabolism,{'original_bigg_ids': ['GTHO']},"{'bigg.reaction': ['GTHOr'], 'biocyc': ['META:..."
NADPHM,,"{'h_c': 1, 'nadp_c': 1, 'nadph_c': -1}",0.0,1000.0,,NaN,NaN,NaN


In [13]:
# metabolite_list=[]
# for mid in glycolysis_ppp_tca_etc.metabolites:
#     metabolite_list.append(mid.id)
# len(metabolite_list)

In [14]:
# met_df=pd.DataFrame()
# for met in metabolite_list:
#     r3d_met= R3D.metabolites.get_by_id(met)
#     m=metabolite_to_dict(r3d_met)
#     df_2=pd.DataFrame.from_dict(m,orient='index')
#     df_2=df_2.T
#     met_df = pd.concat([df_2, met_df])

# met_df=met_df.set_index('id')
# met_df

In [15]:
csv_met = os.path.join(data_dir,"small_core_met_df.csv")
met_df.to_csv(csv_met)

In [16]:
# rxn_df=pd.DataFrame()
# reactions_not_in_r3d=[]
# for rxn in reaction_list:
#     if rxn in R3D.reactions:
#         r3d_rxn= R3D.reactions.get_by_id(rxn)
#         r=reaction_to_dict(r3d_rxn)
#     #     print(r)

#     elif rxn in core.reactions:
#         reaction = core.reactions.get_by_id(rxn)
#         r=reaction_to_dict(reaction)
#         reactions_not_in_r3d.append(rxn)
#     else:
#         reaction = glycolysis_ppp_tca_etc.reactions.get_by_id(rxn)
#         r=reaction_to_dict(reaction)
#         reactions_not_in_r3d.append(rxn)

        
#     df=pd.DataFrame.from_dict(r,orient='index')
#     df=df.T
#     rxn_df = pd.concat([rxn_df,df ])

# rxn_df=rxn_df.set_index('id')
# rxn_df
# reactions_not_in_r3d

In [19]:
csv_rxn = os.path.join(data_dir,"small_core_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [20]:
json_rxn = os.path.join(data_dir,"small_core_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)   